In [ ]:
"""
把从上交所和深交所下载的年报pdf，提取文本

为了加速，采用多进程并发方式
"""

In [12]:
#!jupyter nbconvert --to script pdf_to_txt.ipynb
import os

def getSize(txt_filename):
    fsize=0
    try:
        if os.path.exists(txt_filename):
            fsize = os.path.getsize(txt_filename)
            print(f'【进程{os.getpid()}】file {txt_filename} size is {int(fsize/1024)}kb')
    except Exception as e:
        print(f'【进程{os.getpid()}】getsize exception {txt_filename} ',e)
    return fsize

def pdf_to_txt(pdf_file,txt_filename):
    import subprocess
    import os
    try:
        fsize = getSize(txt_filename)
        if not os.path.exists(os.path.dirname(txt_filename)):
            os.mkdirs(os.path.dirname(txt_filename))    
        if fsize > 1024*100:
            print(f'【进程{os.getpid()}】jump...')
            return 
        print(f'【进程{os.getpid()}】开始转换文件：{pdf_file},{txt_filename} size is {int(fsize/1024)}KB')
        fsize = 0
        try:
            if os.path.exists(pdf_file):
                fsize = os.path.getsize(pdf_file)
        except Exception as e:
            return 
#         out_bytes = subprocess.check_output(['which','pdf2txt.py'])
#         pdf2txt=out_bytes.decode("utf-8").replace('\n','')
#         if not pdf2txt:
#             print('pdf2txt.py 不存在，程序退出<c')
#             quit()
#         subprocess.run(["python", pdf2txt, "-o",txt_filename ,pdf_file])
        t0 = time.time()
        subprocess.run(["pdf2txt.py", "-o",txt_filename ,pdf_file])
        fsize_ret = getSize(txt_filename)
        print(f'【进程{os.getpid()}】转换文件成功：{pdf_file}，用时={int(time.time()-t0)}秒, 转换速度={int(fsize/1024/(time.time()-t0))}KB/秒,txt大小={int(fsize_ret/1024)}KB,pdf大小={int(fsize/1024)}KB,{fsize_ret/fsize}')
        if fsize < 1024* 20:
            print(f'【进程{os.getpid()}】文件太小{fsize_ret}，删除原文件{pdf_file}')
            os.remove(txt_filename)
            os.remove(pdf_file)
            
    except subprocess.CalledProcessError as e:
        out_bytes = e.output       # Output generated before error
        code      = e.returncode   # Return code
        print("E1"*10,e)
    except Exception as e:
        print("E2"*10,e)
     

    


# pdf_file=pdf_dir+'000780_2006.pdf' 
# pdf_to_txt(pdf_file)

In [24]:
# find files
import os
# 启动四个进程进行处理
# python  down_pdf.py all 3 表示启动三个进程，除以3取摸为1的进行
import sys
process_num=2
YEAR = '2014'
if sys.argv[0].find("ipykernel_launcher.py")<=0:
    if len(sys.argv)>=2:
        YEAR = sys.argv[1]
    if len(sys.argv)>=3:
        process_num=int(sys.argv[2])
    
pdf_base_dir ='../pdf/'
txt_dir='../txt/'

# 方法一：扫描pdf目录
# def find_files(pdf_base_dir):
#     pdf_files = []
#     for f in os.listdir(pdf_base_dir):
#         if not(YEAR=='all' or f in YEAR):
#             continue
#         pdf_dir = os.path.join(pdf_base_dir,f)
#         pdf_files += [os.path.join(pdf_dir,f) for f in os.listdir(pdf_dir) if f.endswith('.pdf')]
#     return pdf_files

# pdf_files = find_files(pdf_base_dir)
# print('this time process file num=',len(pdf_files))
# print(pdf_files[:4])
# print('process_num=',process_num)

# 方法二：根据最新的pdf_status计算有多少需要加载
import json
pdf_needs = {}
with open('./pdf_status.json') as f:
    pdf_needs = json.load(f) # {'123456_2001':[pdf,txt,http1,http2,http3]}

pdf_files = []
counter = 0
for code_year,data in pdf_needs.items():
    code,year = code_year.split('_')
    if not(YEAR=='all' or year in YEAR):
        continue 
    if data[1] > 20:
        continue
    if data[1] > 0:      
        print(f"{os.path.join(txt_dir,year,code_year+'.txt')},size={data[1]}KB")
    counter +=1
    pdf_file = os.path.join(pdf_base_dir,year,code_year+'.pdf')
    if os.path.exists(pdf_file):
        pdf_files.append(pdf_file)

pdf_files = sorted(pdf_files,key=lambda x:x[-8:-4],reverse=True)
print(f'condition={YEAR},counter={counter},exists pdf files={len(pdf_files)}')

condition=2014,counter=1232,exists pdf files=1224


In [25]:
#!jupyter nbconvert --to script  --output=
# 移动到临时目录,然后拷贝到其他主机进行并行处理
import shutil
for x in pdf_files:
    d = x.replace('/pdf/','/pdf_tmp/')
    if not os.path.exists(os.path.dirname(d)):
        os.mkdir(os.path.dirname(d))
    shutil.copyfile(x,d)

In [ ]:

import time  
from multiprocessing import Process, Queue
from tqdm import tqdm

q = Queue()

def f(q: Queue):
    while True:
        pdf_file = q.get()
        pdf_to_txt(pdf_file,pdf_file.replace('pdf','txt',3))



            
def loop_task():
    import time  
    for pdf_file in tqdm(pdf_files):
        print("add",pdf_file)
        if q.qsize()> 100:
            print("sleep for queue is not empty")
            time.sleep(10)  # sleep 10秒
        else:
            q.put(pdf_file)

    
for i in range(process_num):
    p = Process(target=f, args=(q,))
    p.start()

loop_task()

    